<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ApogupYVyWlOIro1p6z-qBq42TMttsFI?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)




# Day 1 — AI AGENT BOOTCAMP
The notebook below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool.It has conversational memory - meaning that it can be used as a multi-turn chatbot.

## Agents 🤖

 AI agent is an autonomous software system that can reason, plan, and take action to achieve a specific goal, much like a digital employee. Unlike simple chatbots that just follow a script, an AI agent can proactively break down complex tasks, gather necessary information, and execute the steps to reach a solution without constant human supervision




## Setup and Installation of required libraries
We'll start by installing the prerequisite libraries that we'll be using in this example.

In [ ]:
!pip install -q langchain-openai langgraph langchain-tavily

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 29.3 MB/s eta 0:00:00


## Setup of search tool using Tavily
Agent needs some tools to interact with real world data or environmnet, so let us setup Tavily search tool which can search any query over web and that result can be used by agent

In [ ]:
from langchain_tavily import TavilySearch
import os
from google.colab import userdata
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

search = TavilySearch(max_results=2)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]


In [ ]:
search_results = search.invoke("What is the weather in SF") # testing with a test query, that how our tool is working
for r in search_results["results"]: # printing the response
    print("Title:", r["title"])
    print("URL:", r["url"])
    print("Score:", r["score"])
    print("Content:", r["content"][:100], "...")  # print only first 100 chars
    print()

Title: Weather in San Francisco
URL: https://www.weatherapi.com/
Score: 0.9754234
Content: {'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America' ...

Title: Weather in San Francisco in September 2025
URL: https://world-weather.info/forecast/usa/san_francisco/september-2025/
Score: 0.903098
Content: * Weather in San Francisco # Weather in San Francisco in September 2025 * +73° +72° +59° +68° +61° + ...



## LLM setup
The language model (e.g., OpenAI API) that generates text and reasoning which is the heart of an AI Agent

In [ ]:
from langchain.chat_models import init_chat_model

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
model = init_chat_model("gpt-4.1", model_provider="openai")

In [ ]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! How can I help you today? 😊'

### Creation of agent

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

### Querying to the agent

In [ ]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [ ]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_h7sspoL4TYtca2kh9YlIn3JP)
 Call ID: call_h7sspoL4TYtca2kh9YlIn3JP
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1758031060, 'localtime': '2025-09-16 06:57'}, 'current': {'last_updated_epoch': 1758030300, 'last_updated': '2025-09-16 06:45', 'temp_c': 16.7, 'temp_f': 62.1, 'is

## Adding Memory layer
Adding memory to agent that will make it remember chats which can be refrenced to future messages

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Bob and I live in SF.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob and I live in SF.
================================== Ai Message ==================================

Hi Bob! Nice to meet you. How can I help you today?


In [ ]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_s8TjXT9ppuF1wPDdeomFSKs8)
 Call ID: call_s8TjXT9ppuF1wPDdeomFSKs8
  Args:
    query: current weather in San Francisco, CA
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in San Francisco, CA", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, CA", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1758038979, 'localtime': '2025-09-16 09:09'}, 'current': {'last_updated_epoch': 1758038400, 'last_updated': '2025-09-16 09:


---
### Conclusion
That's a wrap! In this quick start we covered how to create a simple agent, just keep in mind whenever writing prompt Be explicit about **format** and Keep tool descriptions short and precise so the agent knows when to call them




### Next steps (Day 1 -> Day 2)

- Try extending this agent with a small, read-only web-scraping tool (requires API or internet access).
- Day 2: multi-agent patterns (Crew-like crews) and task decomposition.